In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('dataset.csv')
df.drop_duplicates(inplace=True)

In [5]:
# Définir les tranches de BMI
bins = [0, 18.5, 24.9, 29.9, 40, 100]  # Tranches de BMI
labels = ['Sous-poids', 'Poids normal', 'Surpoids', 'Obésité', 'Obésité sévère']

# Ajouter une nouvelle colonne dans le DataFrame pour les tranches de BMI
df['BMI_category'] = pd.cut(df['bmi'], bins=bins, labels=labels, right=False)

In [6]:
# Définir les tranches d'ages
bins = [18, 36, 45, 55, 65]  # Tranches de BMI
labels = ['18-35', '36-45', '46-55', 'plus de 55 ans']

# Ajouter une nouvelle colonne dans le DataFrame pour les tranches de BMI
df['age_categories'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

In [7]:
#Label encoder pour le sexe et si les usagers sont fumeurs

labe_encod = LabelEncoder()
df['sex_encode']= labe_encod.fit_transform(df['sex'])
df['smoker_encode']= labe_encod.fit_transform(df['smoker'])
df

,age,sex,bmi,children,smoker,region,charges,BMI_category,age_categories,sex_encode,smoker_encode
0,19,female,27.900,0,yes,southwest,16884.92400,Surpoids,18-35,0,1
1,18,male,33.770,1,no,southeast,1725.55230,Obésité,18-35,1,0
2,28,male,33.000,3,no,southeast,4449.46200,Obésité,18-35,1,0
3,33,male,22.705,0,no,northwest,21984.47061,Poids normal,18-35,1,0
4,32,male,28.880,0,no,northwest,3866.85520,Surpoids,18-35,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,Obésité,46-55,1,0
1334,18,female,31.920,0,no,northeast,2205.98080,Obésité,18-35,0,0
1335,18,female,36.850,0,no,southeast,1629.83350,Obésité,18-35,0,0
1336,21,female,25.800,0,no,southwest,2007.94500,Surpoids,18-35,0,0


In [8]:
df_reg_encod = pd.get_dummies(df['region'])
df_reg_encod

,northeast,northwest,southeast,southwest
0,False,False,False,True
1,False,False,True,False
2,False,False,True,False
3,False,True,False,False
4,False,True,False,False
...,...,...,...,...
1333,False,True,False,False
1334,True,False,False,False
1335,False,False,True,False
1336,False,False,False,True


In [9]:
# encodage par région et par bmi category

df_reg_encod = pd.get_dummies(df['region'])
df_reg_encod.replace(False, 0, inplace=True)
df_reg_encod.replace(True, 1, inplace=True)
df_reg_encod

df_bmi_encod = pd.get_dummies(df['BMI_category'])
df_bmi_encod.replace(False, 0, inplace=True)
df_bmi_encod.replace(True, 1, inplace=True)
df_bmi_encod

C:\Users\DELL 7490\AppData\Local\Temp\ipykernel_12444\2260671963.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_reg_encod.replace(True, 1, inplace=True)
C:\Users\DELL 7490\AppData\Local\Temp\ipykernel_12444\2260671963.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_bmi_encod.replace(True, 1, inplace=True)


,Sous-poids,Poids normal,Surpoids,Obésité,Obésité sévère
0,0,0,1,0,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,1,0,0,0
4,0,0,1,0,0
...,...,...,...,...,...
1333,0,0,0,1,0
1334,0,0,0,1,0
1335,0,0,0,1,0
1336,0,0,1,0,0


In [10]:
#concatener les df encodé avec le df

df_2 = pd.concat([df,df_reg_encod,df_bmi_encod], axis=1)
#supprimer les colonnes qui ont été encodés

df_2.drop(columns=['sex','smoker','region','BMI_category','age_categories'],inplace=True)
df_2

,age,bmi,children,charges,sex_encode,smoker_encode,northeast,northwest,southeast,southwest,Sous-poids,Poids normal,Surpoids,Obésité,Obésité sévère
0,19,27.900,0,16884.92400,0,1,0,0,0,1,0,0,1,0,0
1,18,33.770,1,1725.55230,1,0,0,0,1,0,0,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,0,1,0,0,0,0,1,0
3,33,22.705,0,21984.47061,1,0,0,1,0,0,0,1,0,0,0
4,32,28.880,0,3866.85520,1,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,0,1,0,0,0,0,0,1,0
1334,18,31.920,0,2205.98080,0,0,1,0,0,0,0,0,0,1,0
1335,18,36.850,0,1629.83350,0,0,0,0,1,0,0,0,0,1,0
1336,21,25.800,0,2007.94500,0,0,0,0,0,1,0,0,1,0,0


In [33]:
line_reg = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(df_2.drop('charges', axis = 1), df_2['charges'], test_size=0.2, shuffle=False)
# line_reg.fit(df_2.drop(['charges', 'children', 'northeast','northwest','southeast','southwest'], axis=1), df_2['charges'])
line_reg.fit(X_train,y_train)
line_reg.coef_

array([  258.54496475,   177.72416666,   427.23507369,  -250.71249616,
       23678.1262052 ,   706.42604595,   190.49311172,  -312.75847279,
        -584.16068489, -1179.958599  , -1112.48730997,  -572.39647927,
        1713.82442071,  1151.01796753])

In [35]:
# line_reg.score(df_2.drop(['charges', 'children', 'northeast','northwest','southeast','southwest'], axis=1), df_2['charges'])
line_reg.score(X_train,y_train)

0.7523006441515494

In [38]:
from sklearn.metrics import mean_squared_error,r2_score

y_pred = line_reg.predict(X_test)

print(f"Mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred):.5f}")

Mean squared error: 6023.32
Coefficient of determination: 0.76430


### regression linéaire sans les fumeurs

In [43]:
# test des resultats si on enleve les fumeurs

df_sans_fumeurs = df_2[df['smoker_encode'] == 0].reset_index()
df_sans_fumeurs.drop('index', axis=1, inplace=True)

In [44]:
line_reg = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(df_sans_fumeurs.drop('charges', axis = 1), df_sans_fumeurs['charges'], test_size=0.2, shuffle=False)
# line_reg.fit(df_2.drop(['charges', 'children', 'northeast','northwest','southeast','southwest'], axis=1), df_2['charges'])
line_reg.fit(X_train,y_train)
line_reg.coef_
y_pred = line_reg.predict(X_test)

print(f"Mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred):.5f}")

Mean squared error: 4307.53
Coefficient of determination: 0.37687


Dans un modèle sans fumeur tous paramètres n'expliquent pas tous 

### regression linéaire avec la seule variable explicative des fumeurs

In [45]:
# test des resultats avec la seule variable explicative des fumeurs
df_smoke = df_2[['charges', 'smoker_encode']]
df_smoke

,charges,smoker_encode
0,16884.92400,1
1,1725.55230,0
2,4449.46200,0
3,21984.47061,0
4,3866.85520,0
...,...,...
1333,10600.54830,0
1334,2205.98080,0
1335,1629.83350,0
1336,2007.94500,0


In [46]:
line_reg = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(df_smoke.drop('charges', axis = 1), df_smoke['charges'], test_size=0.2, shuffle=False)
# line_reg.fit(df_2.drop(['charges', 'children', 'northeast','northwest','southeast','southwest'], axis=1), df_2['charges'])
line_reg.fit(X_train,y_train)
line_reg.coef_
y_pred = line_reg.predict(X_test)

print(f"Mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred):.5f}")

Mean squared error: 7343.52
Coefficient of determination: 0.64966


avec la variable seule si l'assuré est fumeur ou non donne un modèle moins performant donc avec une combinaison de variable différent le modèle est meilleur

### test d'un modele sans les villes

In [51]:
df_sans_ville = df_2.drop(['children', 'northeast','northwest','southeast','southwest'], axis=1)
df_sans_ville

,age,bmi,charges,sex_encode,smoker_encode,Sous-poids,Poids normal,Surpoids,Obésité,Obésité sévère
0,19,27.900,16884.92400,0,1,0,0,1,0,0
1,18,33.770,1725.55230,1,0,0,0,0,1,0
2,28,33.000,4449.46200,1,0,0,0,0,1,0
3,33,22.705,21984.47061,1,0,0,1,0,0,0
4,32,28.880,3866.85520,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,10600.54830,1,0,0,0,0,1,0
1334,18,31.920,2205.98080,0,0,0,0,0,1,0
1335,18,36.850,1629.83350,0,0,0,0,0,1,0
1336,21,25.800,2007.94500,0,0,0,0,1,0,0


In [68]:
line_reg = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(df_sans_ville.drop('charges', axis = 1), df_sans_ville['charges'], test_size=0.2, shuffle=False)
line_reg.fit(X_train,y_train)
line_reg.coef_
y_pred = line_reg.predict(X_test)

print(f"Mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred):.5f}")


Mean squared error: 6056.02
Coefficient of determination: 0.76174


le modèle est sensiblement meilleur avec les villes

### séparation des villes en est et ouest

In [72]:
df_2_est_ouest = df_2.copy()
df_2_est_ouest['ouest'] = df['region'].apply(lambda x: 1 if 'west' in str(x) else 0)
df_2_est_ouest.drop(['northeast','northwest','southeast','southwest'], axis = 1, inplace = True)
df_2_est_ouest

,age,bmi,children,charges,sex_encode,smoker_encode,Sous-poids,Poids normal,Surpoids,Obésité,Obésité sévère,ouest
0,19,27.900,0,16884.92400,0,1,0,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,0,0,1,0,0
2,28,33.000,3,4449.46200,1,0,0,0,0,1,0,0
3,33,22.705,0,21984.47061,1,0,0,1,0,0,0,1
4,32,28.880,0,3866.85520,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,0,0,0,1,0,1
1334,18,31.920,0,2205.98080,0,0,0,0,0,1,0,0
1335,18,36.850,0,1629.83350,0,0,0,0,0,1,0,0
1336,21,25.800,0,2007.94500,0,0,0,0,1,0,0,1


### MEILLEUR MODELE ACTUEL

In [73]:
line_reg = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(df_2_est_ouest.drop('charges', axis = 1), df_2_est_ouest['charges'], test_size=0.2, shuffle=False)
line_reg.fit(X_train,y_train)
line_reg.coef_
y_pred = line_reg.predict(X_test)

print(f"Mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred):.5f}")

Mean squared error: 6017.42
Coefficient of determination: 0.76476


### MEILLEUR MODELE ACTUEL

###==================================================####

### LASSO, en gardant le df_est_ouest qui a donné le meilleur modèle

In [81]:
StandardScaler().ouest

age                 int64
bmi               float64
children            int64
charges           float64
sex_encode          int64
smoker_encode       int64
Sous-poids          int64
Poids normal        int64
Surpoids            int64
Obésité             int64
Obésité sévère      int64
ouest               int64
dtype: object

In [93]:
#MISE EN FORME pour garder les valeurs numériques et centré reduire sur une pipeline
model = make_pipeline(StandardScaler(), PolynomialFeatures(2), Lasso())

X_train, X_test, y_train, y_test = train_test_split(df_2_est_ouest.drop('charges', axis = 1), df_2_est_ouest['charges'], test_size=0.2, shuffle=False)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print(f"Mean squared error: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred):.5f}")

Mean squared error: 4556.84
Coefficient of determination: 0.86510


c:\Users\DELL 7490\Desktop\Projet data\Assur_aimant\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.028e+09, tolerance: 1.547e+07
  model = cd_fast.enet_coordinate_descent(


In [88]:
model[-1].coef_

array([3607.48338155, 1101.54799386,  579.02553818,  -64.72243911,
       9617.19203575, -140.66301305, -409.70022489, -458.60867613,
        698.52401916,  108.39613551, -120.70315129])